In [2]:
#初始化
import os
from openai import AzureOpenAI
#import pypandoc

#pypandoc.download_pandoc()

client = AzureOpenAI(
  api_key = "36a0227eb5954c4cb7e102593a3d78ec",  
  api_version = "2024-06-01",
  azure_endpoint = "https://haxuaoaieus.openai.azure.com/"
)

# 设置openai API默认参数, name = gpt-35-turbo-1106
def get_completion(prompt): 
    messages = [
        {"role": "system", "content": """Translate this markdown content to Simplified Chinese. Follow the below rules:
         1. Must keep original markdown format and position. 
         2. Do not change any markdown or html tags.
         3. The expected documentation or material shall keep the integrality of contents.
         4. The translated content shall be grammatically correct.
         5. The translated content shall be fluent and easy to understand."""},
        {"role": "user", "content": prompt}
        ]
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=messages
      )
    return response.choices[0].message.content


In [3]:
#汉化文件夹中所有md文件
def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def save_to_markdown(file_path, content):
    base_name = os.path.basename(file_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_chs{ext}"
    new_path = os.path.join(os.path.dirname(file_path), new_name)
    with open(new_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_and_save_all_markdown_files(directory_path):
    processed_files_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                content = read_markdown_file(file_path)
                processed_content = get_completion(content)
                save_to_markdown(file_path, processed_content)
                processed_files_count += 1
    return processed_files_count

directory_path = './'
processed_files_count = process_and_save_all_markdown_files(directory_path)

In [7]:

importfile = "./README.md"
savemd = "./README_chs.md"
savedocx = "开发提示词.docx"

def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

markdown_content = read_markdown_file(importfile)

prompt = f"""
    markdown content :{markdown_content}
    """
translated = get_completion(prompt)

def save_to_markdown(file_path, content):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

#save_to_markdown(savemd, translated)

def append_to_md_file(md_file_path, text_to_append):
    with open(md_file_path, 'a', encoding='utf-8') as md_file:
        md_file.write(text_to_append)

append_to_md_file(savemd, translated)

In [ ]:

def convert_md_to_docx(md_file_path, docx_file_path):
    output = pypandoc.convert_file(md_file_path, 'docx', outputfile=docx_file_path)
    assert output == ""

convert_md_to_docx(savemd, savedocx)

In [8]:
##翻译ipynb文件中的所有markdown单元格
import os
import nbformat

def process_and_save_all_ipynb_files(directory_path):
    processed_files_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.ipynb'):
                file_path = os.path.join(root, file)
                process_and_save_ipynb(file_path)
                processed_files_count += 1
    return processed_files_count

def process_and_save_ipynb(file_path):
    # 读取.ipynb文件
    with open(file_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    # 遍历所有单元格
    for i, cell in enumerate(nb.cells):
        # 如果单元格是Markdown类型
        if cell.cell_type == 'markdown':
            # 使用get_completion函数处理单元格内容
            cell.source = get_completion(cell.source)

    # 生成新的文件名
    base_name = os.path.basename(file_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_chs{ext}"
    new_path = os.path.join(os.path.dirname(file_path), new_name)

    # 保存.ipynb文件
    with open(new_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

# 使用函数
directory_path = '../CopilotHackathon/'
processed_files_count = process_and_save_all_ipynb_files(directory_path)